##### Imports

In [2]:
from pathlib import Path
from opi.input.blocks import  BlockGeom, BlockIrc 
import pandas as pd
from comp_chem_utils.utils import show_structure, setup_calc, setup_dock_calc

## BnOH + H2O2

In [12]:
# Define SMILES for host and guest
host = "c1ccccc1CO"  # Benzyl alcohol
guest = "OO"         # Hydrogen peroxide

# Define base working directory
base_name = "BnOH_H2O2"
base_dir = Path(base_name)
xtb_dir = base_dir / "xtb"

### Dock reactants

In [14]:
# Set up docking calculation
calc = setup_dock_calc(
    working_dir=xtb_dir,
    host_smiles=host,
    guest_smiles=guest,
    basename=base_name,
    visualize=False,
)

In [15]:
# run the docking calculation
calc.run()
if calc.get_output().terminated_normally():
    print("Calculation completed successfully.")
    dock_geom = xtb_dir / "dock" / str(base_name + ".docker.xyz")
    show_structure(dock_geom, title=base_name + "Dock Geometry")
else:
    print("Calculation did not complete successfully.")

Calculation completed successfully.


### BnOH_H2O2Dock Geometry

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### TS search

#### TS guess

In [22]:
# Set up dir
ts_dir = xtb_dir / "ts"
ts_guess_dir  = ts_dir / "guess"

# Setup calculation 
geom_block = BlockGeom()
geom_block.constraints = BlockGeom.constraint_str(["a 6 13 17 170.0 C", "B 13 17 2.0 C" , "a 13 17 16 170.0 c"])
geom_block.recalc_hess = 100
geom_block.maxiter = 1000
sk_list = ["xtb", "alpb(water)", "opt", "TightSCF", "Slowconv"]

calc = setup_calc(
    working_dir=ts_guess_dir,
    structure=str(dock_geom),
    basename=base_name + "_ts_guess",
    sk_list=sk_list,
    bk_list=geom_block,
)

### BnOH_H2O2_ts_guess

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [23]:
# run calculation
calc.run()
if calc.get_output().terminated_normally():
    print("Calculation completed successfully.")
    ts_guess_geom = ts_guess_dir / str(base_name + "_ts_guess.xyz")
    show_structure(ts_guess_geom, title=base_name + " TS Guess Geometry")
else:
    print("Calculation did not complete successfully.")

Calculation completed successfully.


### BnOH_H2O2 TS Guess Geometry

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

#### TS Scan

In [25]:
# setup dir
ts_dir = xtb_dir / "ts"
ts_scan_dir  = ts_dir / "scan_ts"

# Setup calculation 
geom_block = BlockGeom()
scan = BlockGeom.scan_str("B 13 16 = 2.0, 0.96, 20")
sk_list = ["xtb", "alpb(water)", "scanTS" , "slowconv", "TightSCF"]
geom_block.scan = scan
geom_block.maxiter = 1000
geom_block.cartfallback = True

calc = setup_calc(
    working_dir=ts_scan_dir,
    structure=str(ts_guess_geom),
    basename= base_name + "_ts_scan",
    sk_list=sk_list,
    bk_list=geom_block,
)

### BnOH_H2O2_ts_scan

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# run calculation
calc.run()
if calc.get_output().terminated_normally():
    print("Calculation completed successfully.")
    for file in ts_scan_dir.glob("*.refined.xyz"):
      ts_geom = str(file).split('.')[0] + '.' + str(file).split('.')[1] + ".xyz"
      show_structure(ts_geom, title="TS scan result")
else:
    print("Calculation did not complete successfully.")

Calculation completed successfully.


### BnOH_H2O2 TS Scan Geometry

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

#### TS opt

In [29]:
# ts_opt dir
ts_dir = xtb_dir / "ts"
ts_opt_dir  = ts_dir / "ts_opt"

# Setup calculation
geom_block = BlockGeom()
irc_block = BlockIrc(maxiter=100)
sk_list = ["xtb", "alpb(water)", "optTS" , "slowconv", "TightSCF", "freq", "IRC"]
geom_block.maxiter = 1000
geom_block.recalc_hess = 25
geom_block.calc_hess = True
geom_block.cartfallback = True
geom_path =ts_geom 

calc = setup_calc(
    working_dir=ts_opt_dir,
    structure=geom_path,
    basename=base_name + "_ts_opt",
    sk_list=sk_list,
    bk_list=[geom_block, irc_block],
)

### BnOH_H2O2_ts_opt

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [30]:
# run calculation
calc.run()
if calc.get_output().terminated_normally():
    print("Calculation completed successfully.")
    ts_opt_geom = ts_opt_dir / str(base_name + "_ts_opt.xyz")
    show_structure(ts_opt_geom, title=base_name + " TS Geometry")
else:
    print("Calculation did not complete successfully.")

Calculation completed successfully.


### BnOH_H2O2 TS Geometry

3Dmol.js failed to load for some reason. Please check your browser console for error messages.